In [1]:
import Base: iterate
using Printf, IterativeSolvers, LinearAlgebra, Random, Distributions, Preconditioners
export cg, cg!, CGSampler, PCGSampler, cgSamp!, CGStateVariables

"""
  y ~ N(A^(-1)b , A^(-1))
  x ~ A^(-1)b
"""
mutable struct CGSampler{matT, solT, vecT, numT <: Real}
    A::matT
    x::solT
    r::vecT
    c::vecT
    u::vecT
    y::vecT
    α::Float64
    a::Array{Float64,1}
    b::Array{Complex,1}
    reltol::numT
    residual::numT
    prev_residual::numT
    maxiter::Int
    mv_products::Int
end

mutable struct PCGSampler{precT, matT, solT, vecT, numT <: Real, paramT <: Number}
    Pl::precT
    A::matT
    x::solT
    r::vecT
    c::vecT
    u::vecT
    y::vecT
    α::Float64
    a::Array{Float64,1}
    b::Array{Complex,1}
    reltol::numT
    residual::numT
    ρ::paramT
    maxiter::Int
    mv_products::Int
end

@inline converged(it::Union{CGSampler, PCGSampler}) = it.residual ≤ it.reltol

@inline start(it::Union{CGSampler, PCGSampler}) = 0

@inline done(it::Union{CGSampler, PCGSampler}, iteration::Int) = iteration ≥ it.maxiter || converged(it)


###############
# Ordinary CG #
###############

function iterate(it::CGSampler, iteration::Int=start(it))
    if done(it, iteration) return nothing end
    # u := r + βu (almost an axpy)
    β = it.residual^2 / it.prev_residual^2
    if(iteration != 0) 
        append!( it.a, - β/it.α)
        append!( it.b, sqrt(complex(-β))/it.α )
    end
    it.u .= it.r .+ β .* it.u

    # c = A * u
    mul!(it.c, it.A, it.u)
    α = it.residual^2 / dot(it.u, it.c)
    it.a[iteration+1] =  it.a[iteration+1] + 1/it.α
    # Improve solution and residual
    it.x .+= α .* it.u
    it.y = it.y + randn() .* it.u ./sqrt(dot(it.u, it.c)) 
    it.r .-= α .* it.c
    
    it.prev_residual = it.residual
    it.residual = norm(it.r)

    # Return the residual at item and iteration number as state
    it.residual, iteration + 1
end

#####################
# Preconditioned CG #
#####################

function iterate(it::PCGSampler, iteration::Int=start(it))
    # Check for termination first
    #print(iteration)
    if done(it, iteration)
        return nothing
    end
    ## c = Pl \ r ##
    ldiv!(it.c, it.Pl, it.r)

    ρ_prev = it.ρ
    it.ρ = dot(it.c, it.r)

    # u := c + βu (almost an axpy)
    β = it.ρ / ρ_prev
    if(iteration != 0) 
        append!( it.a, - β/it.α)
        append!( it.b, sqrt(complex(-β))/it.α )
    end
    it.u .= it.c .+ β .* it.u

    # c = A * u
    mul!(it.c, it.A, it.u)
    it.α = it.ρ / dot(it.u, it.c)
    it.a[iteration+1] =  it.a[iteration+1] + 1/it.α
    
    # Improve solution and residual
    it.x .+= it.α .* it.u
    it.y = it.y + randn() .* it.u ./sqrt(dot(it.u, it.c)) 
    it.r .-= it.α .* it.c

    it.residual = norm(it.r)

    # Return the residual at item and iteration number as state
    it.residual, iteration + 1
end

iterate (generic function with 287 methods)

In [2]:
struct CGStateVariables{T,Tx<:AbstractArray{T}}
    u::Tx
    r::Tx
    c::Tx
end

function cg_sampler!(x, A, y0, b, Pl = Identity();
    tol = sqrt(eps(real(eltype(b)))),
    maxiter::Int = size(A, 2),
    statevars::CGStateVariables = CGStateVariables(zero(x), similar(x), similar(x)),
    initially_zero::Bool = false
)
    u = statevars.u
    r = statevars.r
    c = statevars.c
    u .= zero(eltype(x))
    copyto!(r, b)

    # Compute r with an MV-product or not.
    if initially_zero
        mv_products = 0
        c = similar(x)
        residual = norm(b)
        reltol = residual * tol # Save one dot product
    else
        mv_products = 1
        mul!(c, A, x)
        r .-= c
        residual = norm(r)
        reltol = norm(b) * tol
    end

    # Return the Sampler
    if isa(Pl, Identity)
        return CGSampler(A, x, r, c, u, y0, 0., zeros(1), zeros(Complex,0),
            reltol, residual, one(residual),
            maxiter, mv_products
        )
    else
        return PCGSampler(Pl, A, x, r, c, u, y0, 0., zeros(1), zeros(Complex,0),
            reltol, residual, one(eltype(x)),
            maxiter, mv_products
        )
    end
end


cg_sampler! (generic function with 2 methods)

In [3]:
function cgSamp!(x, A, y0, b;
    tol = sqrt(eps(real(eltype(b)))),
    maxiter::Int = size(A, 2),
    log::Bool = false,
    statevars::CGStateVariables = CGStateVariables(zero(x), similar(x), similar(x)),
    verbose::Bool = false,
    Pl = Identity(),
    kwargs...
)
    history = ConvergenceHistory(partial = !log)
    history[:tol] = tol
    log && reserve!(history, :resnorm, maxiter + 1)

    # Actually perform CG
    iterable = cg_sampler!(x, A, y0, b, Pl; tol = tol, maxiter = maxiter, statevars = statevars, kwargs...)
    if log
        history.mvps = iterable.mv_products
    end
    for (iteration, item) = enumerate(iterable)
        if log
            nextiter!(history, mvps = 1)
            push!(history, :resnorm, iterable.residual)
        end
        verbose && @printf("%3d\t%1.2e\n", iteration, iterable.residual)
    end

    verbose && println()
    log && setconv(history, converged(iterable))
    log && shrink!(history)

    log ? (iterable.x, history) : iterable
end


cgSamp! (generic function with 1 method)

In [4]:
include("../src/data_generation.jl")
using Random, LinearAlgebra, Preconditioners, BenchmarkTools


In [5]:
Random.seed!(111)
n = 10
A = gen_A(n)
b = randn(n^2)
x = zeros(n^2)
L = LowerTriangular(A)
p = CholeskyPreconditioner(L, 2)
samp = cgSamp!(x, A, x, b;
    tol = sqrt(eps(real(eltype(b)))),
    maxiter= 50,
    Pl = p)
samp.x

100-element Array{Float64,1}:
 -508.2912457638938 
 -508.9804860526637 
 -508.958887873163  
 -509.9352042344667 
 -510.6453735326395 
 -510.37934186963656
 -509.908469475584  
 -510.2317711822458 
 -509.7419206286623 
 -509.05902017055774
 -508.422397620369  
 -508.6074585988139 
 -509.34587990843613
    ⋮               
 -508.13853687737674
 -508.38457667174526
 -508.6311657322781 
 -508.0445684821186 
 -509.3877032387501 
 -509.2786551853962 
 -509.11393304625585
 -509.2279385251645 
 -509.0197662063185 
 -507.9741640533016 
 -508.2196218355882 
 -509.06992113789573

In [100]:
Random.seed!(111)
n = 10
A = gen_A(n)
b = randn(n^2)
ssor(A, b, 1.6641, maxiter=70000)

100-element Array{Float64,1}:
 -508.2912435681162 
 -508.9804838641082 
 -508.9588856850026 
 -509.9352020460191 
 -510.6453713458184 
 -510.3793396808102 
 -509.9084672879415 
 -510.2317689937005 
 -509.74191843892856
 -509.0590179846583 
 -508.4223954291875 
 -508.6074564077927 
 -509.345877720993  
    ⋮               
 -508.13853468572677
 -508.384574482358  
 -508.63116354176185
 -508.0445662898434 
 -509.38770104832855
 -509.2786529978664 
 -509.1139308588661 
 -509.2279363360354 
 -509.01976401935644
 -507.9741618672869 
 -508.2196196464825 
 -509.0699189471981 

In [125]:
cg!(x, A, b;
    tol = sqrt(eps(real(eltype(b)))),
    maxiter= 100,
    Pl = p)

100-element Array{Float64,1}:
 -508.2912457747934 
 -508.9804860697394 
 -508.95888787324384
 -509.93520423275567
 -510.64537352740535
 -510.3793418651406 
 -509.9084694799371 
 -510.23177118970784
 -509.74192063569416
 -509.0590201796033 
 -508.4223976091782 
 -508.60745860037065
 -509.3458799059738 
    ⋮               
 -508.1385368693835 
 -508.38457666668904
 -508.6311657306854 
 -508.0445684780483 
 -509.38770324174794
 -509.27865518547696
 -509.1139330439906 
 -509.2279385301774 
 -509.01976621585237
 -507.9741640566261 
 -508.21962183658866
 -509.06992113211567

In [45]:
include("../src/cheby_mj.jl")
Random.seed!(111)
n = 10
A = gen_A(n)
b = randn(n^2)
ω=1.6641
M, m = eigMm(A,ω)

cheby_ssor(A, b, ω, M,m, maxiter=100000)
#cheby_ssor_sampler(A, b, ω, M, m,𝛎 =randn(100) , maxiter=100000)

(0.9998564750469746, 0.0002751717871851232)

In [21]:
a=1.
randn!()

MethodError: MethodError: no method matching randn!(::Float64)
Closest candidates are:
  randn!(!Matched::AbstractRNG, !Matched::AbstractArray{T,N} where N) where T at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Random/src/normal.jl:172
  randn!(!Matched::SharedArrays.SharedArray) at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/SharedArrays/src/SharedArrays.jl:530
  randn!(!Matched::AbstractArray) at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Random/src/normal.jl:178

In [15]:

n = 10
A = gen_A(n)
x = zeros(n^2)

L = LowerTriangular(A)
p = CholeskyPreconditioner(L, 2)

CholeskyPreconditioner{Float64,SparseMatrixCSC{Float64,Int64}}([2.0001 0.0 … 0.0 0.0; -1.0 3.0001 … 0.0 0.0; … ; 0.0 0.0 … 3.0001 0.0; 0.0 0.0 … -1.0 2.0001], 2)

In [15]:
include("matrixgen.jl")
A = laplacematrix(512)

262144×262144 SparseMatrixCSC{Float64,Int64} with 1308672 stored entries:
  [1     ,      1]  =  2.0001
  [2     ,      1]  =  -1.0
  [513   ,      1]  =  -1.0
  [1     ,      2]  =  -1.0
  [2     ,      2]  =  3.0001
  [3     ,      2]  =  -1.0
  [514   ,      2]  =  -1.0
  [2     ,      3]  =  -1.0
  [3     ,      3]  =  3.0001
  [4     ,      3]  =  -1.0
  [515   ,      3]  =  -1.0
  [3     ,      4]  =  -1.0
  ⋮
  [262141, 262141]  =  3.0001
  [262142, 262141]  =  -1.0
  [261630, 262142]  =  -1.0
  [262141, 262142]  =  -1.0
  [262142, 262142]  =  3.0001
  [262143, 262142]  =  -1.0
  [261631, 262143]  =  -1.0
  [262142, 262143]  =  -1.0
  [262143, 262143]  =  3.0001
  [262144, 262143]  =  -1.0
  [261632, 262144]  =  -1.0
  [262143, 262144]  =  -1.0
  [262144, 262144]  =  2.0001

In [16]:
L = LowerTriangular(A)

262144×262144 LowerTriangular{Float64,SparseMatrixCSC{Float64,Int64}}:
  2.0001    ⋅        ⋅        ⋅      …    ⋅        ⋅        ⋅       ⋅    
 -1.0      3.0001    ⋅        ⋅           ⋅        ⋅        ⋅       ⋅    
  0.0     -1.0      3.0001    ⋅           ⋅        ⋅        ⋅       ⋅    
  0.0      0.0     -1.0      3.0001       ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0     -1.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0     …    ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0     …    ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0          ⋅        ⋅        ⋅       ⋅    
  0.0      0.0      0.0      0.0         

In [20]:
p = CholeskyPreconditioner(L, 2)
x = zeros(512^2)
b = randn(512^2);

In [21]:
samp = cgSamp!(x, A, x, b;
    tol = sqrt(eps(real(eltype(b)))),
    maxiter= 50,
    Pl = p)
samp.x

262144-element Array{Float64,1}:
  20.705392088565198
  19.955280248837173
  19.676219505442706
  19.78235786522592 
  20.044658020883983
  19.347270962557214
  18.554601815598666
  17.974547408514518
  17.884418234570646
  18.26788571276589 
  17.844190667480415
  17.737743495872582
  18.075605176737763
   ⋮                
 -43.35690856242121 
 -41.97127511694285 
 -40.96219644218682 
 -40.14547433871541 
 -39.055539656324406
 -38.675531519039914
 -38.76193666436958 
 -38.078334697682266
 -37.263672140151535
 -37.313901837139   
 -36.85923396932515 
 -36.778120495713736

In [23]:
samp.b

49-element Array{Complex,1}:
 0.0 + 0.06620089434484114im
 0.0 + 0.06757839735721233im
 0.0 + 0.08159498164038559im
 0.0 + 0.11932533474510076im
 0.0 + 0.1609015541049887im 
 0.0 + 0.1125975012631055im 
 0.0 + 0.08416063748265495im
 0.0 + 0.12419958598963761im
 0.0 + 0.12232445508280095im
 0.0 + 0.0808902566167037im 
 0.0 + 0.08862879216220831im
 0.0 + 0.08359653973198192im
 0.0 + 0.07368596877002291im
     ⋮                      
 0.0 + 0.08708976127173183im
 0.0 + 0.08678210359680892im
 0.0 + 0.08319356000644779im
 0.0 + 0.07901385980509103im
 0.0 + 0.085454375322866im  
 0.0 + 0.11387021913227316im
 0.0 + 0.1304580808226108im 
 0.0 + 0.09478877250763643im
 0.0 + 0.07934341138523987im
 0.0 + 0.07866221550906312im
 0.0 + 0.0794757215925305im 
 0.0 + 0.07820522486640574im

In [14]:

b = randn(n^2)

100-element Array{Float64,1}:
  0.9840081770575446 
  1.5120529400061509 
 -0.7320340092586142 
 -1.5382633913261394 
 -0.0801791185040779 
 -0.21820670010876822
  0.500781551660488  
 -0.06159156197969655
  1.9774842642942712 
  0.3833200357791712 
 -0.6601000297773038 
 -1.0768640096163058 
 -1.0233498786140616 
  ⋮                  
  0.5021309525870198 
 -0.3483132137748249 
 -1.5494142108480256 
  0.13269048942903844
 -0.6814260125895202 
 -0.08663762529155392
  0.7673788652498749 
 -2.3059035073249547 
 -0.6811462758510334 
  0.10393133443104506
 -1.2810640940491917 
  1.4992718805933383 

In [13]:
vcat(Complex.(samp.a), samp.b) 

59-element Array{Complex,1}:
   0.23748329896071782 + 0.0im                 
   0.18349849023768833 + 0.0im                 
   0.06809622578130593 + 0.0im                 
  0.002569988240799209 + 0.0im                 
  -0.04683131830077242 + 0.0im                 
 -0.042238130140678565 + 0.0im                 
 -0.042038269273782986 + 0.0im                 
  -0.07710586095997832 + 0.0im                 
  -0.11218397955079021 + 0.0im                 
  -0.07902816722130268 + 0.0im                 
  -0.07398595873303765 + 0.0im                 
  -0.06009641520340964 + 0.0im                 
   0.10331103628674951 + 0.0im                 
                       ⋮                       
                   0.0 + 0.04691446107521637im 
                   0.0 + 0.04330710120696413im 
                   0.0 + 0.05030946291596315im 
                   0.0 + 0.0758514374423246im  
                   0.0 + 0.1235992213232298im  
                   0.0 + 0.0865310275897783im  
           

In [43]:
function nz_vals(k, a, b)
    # a, b: complex
    # a: k-vector, b: k-1 vector
    nzs = vcat(a[1], b[1])
    ck = vcat(b[k-1], a[k])
    for i in 2:k-1
        colunit = vcat(b[i-1], a[i], b[i])
        nzs = vcat(nzs, colunit)
    end
    nzs = vcat(nzs, ck)
    nzs
end

nz_vals (generic function with 2 methods)

In [25]:
function rowidx(k)
    idx = [1; 2]
    unit = [-1; 0; 1]
    for i in 2:k-1
        idx = vcat(idx, unit.+i)
    end
    idx = vcat(idx, [k-1; k])
    idx
end

rowidx (generic function with 1 method)

In [30]:
k = 3
vcat(2, repeat([3], k-2), 2)

3-element Array{Int64,1}:
 2
 3
 2

In [31]:
function colptr(k)
    seq = vcat(2, repeat([3], k-2), 2)
    temp = ones(Int64, k+1)
    for i in 2:k+1
        temp[i] = temp[i-1] + seq[i-1]
    end
    temp
end

colptr (generic function with 1 method)

In [32]:
colptr(5)

6-element Array{Int64,1}:
  1
  3
  6
  9
 12
 14

In [36]:
using SparseArrays

In [44]:
A = SparseMatrixCSC(50, 50, colptr(50), rowidx(50), nz_vals(50, a, b))

50×50 SparseMatrixCSC{Complex{Float64},Int64} with 148 stored entries:
  [1 ,  1]  =  0.188469+0.0im
  [2 ,  1]  =  0.0+0.0662009im
  [1 ,  2]  =  0.0+0.0662009im
  [2 ,  2]  =  0.0660515+0.0im
  [3 ,  2]  =  0.0+0.0675784im
  [2 ,  3]  =  0.0+0.0675784im
  [3 ,  3]  =  0.0398384+0.0im
  [4 ,  3]  =  0.0+0.081595im
  [3 ,  4]  =  0.0+0.081595im
  [4 ,  4]  =  0.0455474+0.0im
  [5 ,  4]  =  0.0+0.119325im
  [4 ,  5]  =  0.0+0.119325im
  ⋮
  [46, 46]  =  -0.00571468+0.0im
  [47, 46]  =  0.0+0.0793434im
  [46, 47]  =  0.0+0.0793434im
  [47, 47]  =  0.00189459+0.0im
  [48, 47]  =  0.0+0.0786622im
  [47, 48]  =  0.0+0.0786622im
  [48, 48]  =  0.00251943+0.0im
  [49, 48]  =  0.0+0.0794757im
  [48, 49]  =  0.0+0.0794757im
  [49, 49]  =  0.0016515+0.0im
  [50, 49]  =  0.0+0.0782052im
  [49, 50]  =  0.0+0.0782052im
  [50, 50]  =  0.00238808+0.0im

In [46]:
a

50-element Array{Complex{Float64},1}:
    0.18846874281103776 + 0.0im
    0.06605148509874463 + 0.0im
   0.039838359455287034 + 0.0im
    0.04554740615978575 + 0.0im
    0.05409249743758619 + 0.0im
  -0.007402659057657579 + 0.0im
    0.00377210910298717 + 0.0im
    0.03770435485724731 + 0.0im
   0.015015561166412422 + 0.0im
  -0.012381939145657994 + 0.0im
   0.014830627333642293 + 0.0im
   0.004002240797597437 + 0.0im
 -0.0004966106106656515 + 0.0im
                        ⋮      
   0.003001425830882684 + 0.0im
   0.000759111710341806 + 0.0im
 0.00026924501407811496 + 0.0im
   0.005810706302993449 + 0.0im
    0.01754492736271432 + 0.0im
   0.014836399354218321 + 0.0im
  -0.020038390376414583 + 0.0im
  -0.005714677973388885 + 0.0im
  0.0018945906538230528 + 0.0im
  0.0025194316040171444 + 0.0im
  0.0016515023564118159 + 0.0im
  0.0023880817573888186 + 0.0im

In [50]:
using Arpack
M = eigs(A; nev=1, ritzvec=false, which=:LM)[1][1]


0.023603549696475008 + 0.22780925450536535im

In [ ]:
m = eigs(A; nev=1, ritzvec=false, which=:SM)[1][1]

In [33]:
a = Complex.(samp.a)
b = samp.b


49-element Array{Complex,1}:
 0.0 + 0.06620089434484114im
 0.0 + 0.06757839735721233im
 0.0 + 0.08159498164038559im
 0.0 + 0.11932533474510076im
 0.0 + 0.1609015541049887im 
 0.0 + 0.1125975012631055im 
 0.0 + 0.08416063748265495im
 0.0 + 0.12419958598963761im
 0.0 + 0.12232445508280095im
 0.0 + 0.0808902566167037im 
 0.0 + 0.08862879216220831im
 0.0 + 0.08359653973198192im
 0.0 + 0.07368596877002291im
     ⋮                      
 0.0 + 0.08708976127173183im
 0.0 + 0.08678210359680892im
 0.0 + 0.08319356000644779im
 0.0 + 0.07901385980509103im
 0.0 + 0.085454375322866im  
 0.0 + 0.11387021913227316im
 0.0 + 0.1304580808226108im 
 0.0 + 0.09478877250763643im
 0.0 + 0.07934341138523987im
 0.0 + 0.07866221550906312im
 0.0 + 0.0794757215925305im 
 0.0 + 0.07820522486640574im

In [10]:
samp.a
samp.b

29-element Array{Complex,1}:
 0.0 + 0.08464928969031496im 
 0.0 + 0.15442592708168015im 
 0.0 + 0.14118301805995154im 
 0.0 + 0.1254188600571702im  
 0.0 + 0.12077248279400081im 
 0.0 + 0.1175351624789117im  
 0.0 + 0.09828489559274567im 
 0.0 + 0.08603066204976287im 
 0.0 + 0.06956212594323452im 
 0.0 + 0.07619404225381202im 
 0.0 + 0.07503841330506228im 
 0.0 + 0.055657164238697605im
 0.0 + 0.06735699406414618im 
     ⋮                       
 0.0 + 0.04691446107521637im 
 0.0 + 0.04330710120696413im 
 0.0 + 0.05030946291596315im 
 0.0 + 0.0758514374423246im  
 0.0 + 0.1235992213232298im  
 0.0 + 0.0865310275897783im  
 0.0 + 0.046123644821078245im
 0.0 + 0.041569699316312846im
 0.0 + 0.05963682730164468im 
 0.0 + 0.06592255825973697im 
 0.0 + 0.05869614019492629im 
 0.0 + 0.05147885756106756im 